In [ ]:
import pandas as pd
import numpy as np
import requests
from lxml import html
import xml.etree.ElementTree as et
from decimal import Decimal

In [ ]:
# ------------------ GET PRODUCT URLs
# define url - site map of all the products
url = 'https://www.rohlik.cz/sitemap.xml'

# create http response object
resp = requests.get(url)

# parse xml
tree = et.fromstring(resp.content)
child = tree.getchildren()

# extract product URLs
products = [i[0].text for i in child]

# convert to panda series (it supports regex)
products = pd.Series(products)

# use regex to extract URLs that contain products with their prices - defined as 7 numbers and text, divided by '-'
products = products[products.str.contains('https://www.rohlik.cz/[0-9]{1,8}-*')]

In [ ]:
products[:10]